### Hybrid Recommender System Project

**İş Problemi**

ID'si verilen kullanıcı için item-based ve user-based recommender yöntemlerini kullanarak 10 film önerisi yapınız.

5 öneri user-based modelden 5 öneri de item-based modelden ele alınız ve nihai olarak 10 öneriyi 2 modelden yapınız.

**Veri Seti Hikayesi**

Veri seti, bir film tavsiye hizmeti olan MovieLens tarafından sağlanmıştır. İçerisinde filmler ile birlikte bu filmlere yapılan derecelendirme puanlarını barındırmaktadır. 27.278 filmde 2.000.0263 derecelendirme içermektedir. Bu veri seti ise 17 Ekim 2016 tarihinde oluşturulmuştur. 138.493 kullanıcı ve 09 Ocak 1995 ile 31 Mart 2015 tarihleri arasında verileri içermektedir. Kullanıcılar rastgele seçilmiştir. Seçilen tüm kullanıcıların en az 20 filme oy verdiği bilgisi mevcuttur.

**Değişkenler**

**movieId:** Eşsiz film numarası

**title:** Film adı

**genres:** Tür

**userId:** Eşisiz kullanıcı numarası (UniqueID)

**movieId:** Eşsiz film numarası (UniqueID)

**rating:** Kullanıcı tarafından filme verilen puan

**timestamp** Değerlendirme tarihi

#### Görev 1: Verinin Hazırlanması

**Adım 1:** movie, rating veri setlerini okutunuz.

In [1]:
# Gerekli kütüphane ve fonksiyonların yüklenmesi

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)

movie = pd.read_csv("datasets/movie.csv")

movie.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
movie.shape

(27278, 3)

In [3]:
rating = pd.read_csv("datasets/rating.csv")

rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [4]:
rating.shape

(20000263, 4)

In [5]:
rating["userId"].nunique()

138493

**Adım 2:** rating veri setine Id’lere ait film isimlerini ve türünü movie veri setinden ekleyiniz.

In [6]:
df = movie.merge(rating, how = "left", on = "movieId")

df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [7]:
df.shape

(20000797, 6)

**Adım 3:** Herbir film için toplam kaç kişinin oy kullanıdığını hesaplayınız. Toplam oy kullanılma sayısı 1000'in altında olan filmlerin isimlerini listede tutunuz ve veri setinden çıkartınız.

In [8]:
# Her bir film için toplam kaç kişinin oy kullandığını hesaplıyoruz.

comment_counts = pd.DataFrame(df["title"].value_counts())

comment_counts

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [9]:
# Toplam oy kullanma sayısı 1000'in altında olan filmlerin isimlerini rare_movies de tutuyoruz.
# Ve veri setinden çıkarıyoruz.

In [10]:
rare_movies = comment_counts[comment_counts["title"] <= 1000].index

common_movies = df[~ df["title"].isin(rare_movies)]

common_movies.shape

(17766015, 6)

**Adım 4:** index'te userID'lerin sutunlarda film isimlerinin ve değer olarak ratinglerin bulunduğu dataframe için pivot table oluşturunuz.

In [11]:
common_movies.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [12]:
user_movie_df = common_movies.pivot_table(index = ["userId"], columns = ["title"], values = "rating")

user_movie_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),3-Iron (Bin-jip) (2004),30 Days of Night (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",42 Up (1998),48 Hrs. (1982),50 First Dates (2004),50/50 (2011),54 (1998),"6th Day, The (2000)","7th Voyage of Sinbad, The (1958)",8 1/2 (8½) (1963),8 Heads in a Duffel Bag (1997),8 Mile (2002),8MM (1999),9 (2009),9 1/2 Weeks (Nine 1/2 Weeks) (1986),"A-Team, The (2010)",A.I. Artificial Intelligence (2001),AVP: Alien vs. Predator (2004),AVPR: Aliens vs. Predator - Requiem (2007),About Last Night... (1986),About Schmidt (2002),About a Boy (2002),"Absent-Minded Professor, The (1961)",Absolute Power (1997),"Abyss, The (1989)",Accepted (2006),"Accidental Tourist, The (1988)","Accused, The (1988)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Universe (2007),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)",Addicted to Love (1997),"Adjustment Bureau, The (2011)",Adventureland (2009),Adventures in Babysitting (1987),"Adventures of Baron Munchausen, The (1988)","Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)","Adventures of Milo and Otis, The (Koneko monogatari) (1986)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Adventures of Rocky and Bullwinkle, The (2000)","Adventures of Tintin, The (2011)",Aeon Flux (2005),"Affair to Remember, An (1957)",Affliction (1997),"African Queen, The (1951)",After Hours (1985),"Age of Innocence, The (1993)",Agnes of God (1985),"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)",Air America (1990),Air Bud (1997),Air Force One (1997),Airheads (1994),Airplane II: The Sequel (1982),Airplane! (1980),Airport (1970),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alexander (2004),Alfie (2004),Ali (2001),Alice in Wonderland (1951),Alice in Wonderland (2010),Alien (1979),Alien Nation (1988),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Alive (1993),All About Eve (1950),All About My Mother (Todo sobre mi madre) (1999),All Dogs Go to Heaven (1989),All Dogs Go to Heaven 2 (1996),All Quiet on the Western Front (1930),All That Jazz (1979),All the President's Men (1976),All the Pretty Horses (2000),Almost Famous (2000),Along Came Polly (2004),Along Came a Spider (2001),"Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)",Altered States (1980),Always (1989),Amadeus (1984),Amateur (1994),"Amazing Spider-Man, The (2012)","Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",America's Sweethearts (2001),American Beauty (1999),American Gangster (2007),American Gigolo (1980),American Graffiti (1973),American History X (1998),American Hustle (2013),American Movie (1999),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),American Splendor (2003),"American Tail, An (1986)","American Tail: Fievel Goes West, An (1991)",Amer

**Adım 5:** Yapılan tüm işlemleri fonksiyonlaştırınız.

In [13]:
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('datasets/movie.csv')
    rating = pd.read_csv('datasets/rating.csv')
    df = movie.merge(rating, how = "left", on = "movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~ df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index = ["userId"], columns = ["title"], values = "rating")
    return user_movie_df

In [14]:
user_movie_df = create_user_movie_df()

**Görev 2:** Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

**Adım 1:** Rastgele bir kullanıcı id’si seçiniz.

In [15]:
random_user = 108170

**Adım 2:** Seçilen kullanıcıya ait gözlem birimlerinden oluşan random_user_df adında yeni bir dataframe oluşturunuz.

In [16]:
random_user_df = user_movie_df[user_movie_df.index == random_user]

random_user_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),3-Iron (Bin-jip) (2004),30 Days of Night (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",42 Up (1998),48 Hrs. (1982),50 First Dates (2004),50/50 (2011),54 (1998),"6th Day, The (2000)","7th Voyage of Sinbad, The (1958)",8 1/2 (8½) (1963),8 Heads in a Duffel Bag (1997),8 Mile (2002),8MM (1999),9 (2009),9 1/2 Weeks (Nine 1/2 Weeks) (1986),"A-Team, The (2010)",A.I. Artificial Intelligence (2001),AVP: Alien vs. Predator (2004),AVPR: Aliens vs. Predator - Requiem (2007),About Last Night... (1986),About Schmidt (2002),About a Boy (2002),"Absent-Minded Professor, The (1961)",Absolute Power (1997),"Abyss, The (1989)",Accepted (2006),"Accidental Tourist, The (1988)","Accused, The (1988)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Universe (2007),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)",Addicted to Love (1997),"Adjustment Bureau, The (2011)",Adventureland (2009),Adventures in Babysitting (1987),"Adventures of Baron Munchausen, The (1988)","Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)","Adventures of Milo and Otis, The (Koneko monogatari) (1986)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Adventures of Rocky and Bullwinkle, The (2000)","Adventures of Tintin, The (2011)",Aeon Flux (2005),"Affair to Remember, An (1957)",Affliction (1997),"African Queen, The (1951)",After Hours (1985),"Age of Innocence, The (1993)",Agnes of God (1985),"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)",Air America (1990),Air Bud (1997),Air Force One (1997),Airheads (1994),Airplane II: The Sequel (1982),Airplane! (1980),Airport (1970),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alexander (2004),Alfie (2004),Ali (2001),Alice in Wonderland (1951),Alice in Wonderland (2010),Alien (1979),Alien Nation (1988),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Alive (1993),All About Eve (1950),All About My Mother (Todo sobre mi madre) (1999),All Dogs Go to Heaven (1989),All Dogs Go to Heaven 2 (1996),All Quiet on the Western Front (1930),All That Jazz (1979),All the President's Men (1976),All the Pretty Horses (2000),Almost Famous (2000),Along Came Polly (2004),Along Came a Spider (2001),"Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)",Altered States (1980),Always (1989),Amadeus (1984),Amateur (1994),"Amazing Spider-Man, The (2012)","Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",America's Sweethearts (2001),American Beauty (1999),American Gangster (2007),American Gigolo (1980),American Graffiti (1973),American History X (1998),American Hustle (2013),American Movie (1999),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),American Splendor (2003),"American Tail, An (1986)","American Tail: Fievel Goes West, An (1991)",Amer

In [17]:
random_user_df.shape

(1, 3159)

**Adım 3:** Seçilen kullanıcıların oy kullandığı filmleri movies_watched adında bir listeye atayınız.

In [18]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].to_list()

movies_watched

['2001: A Space Odyssey (1968)',
 'Adventures of Priscilla, Queen of the Desert, The (1994)',
 'Akira (1988)',
 'Aladdin (1992)',
 'Aliens (1986)',
 'Almost Famous (2000)',
 'Along Came Polly (2004)',
 'Amadeus (1984)',
 'American History X (1998)',
 'Any Given Sunday (1999)',
 'Arachnophobia (1990)',
 'Army of Darkness (1993)',
 'Bad Santa (2003)',
 'Basic Instinct (1992)',
 'Batman Returns (1992)',
 'Big Lebowski, The (1998)',
 'Blair Witch Project, The (1999)',
 'Blues Brothers, The (1980)',
 'Boondock Saints, The (2000)',
 'Bourne Identity, The (2002)',
 'Braveheart (1995)',
 'Breakfast Club, The (1985)',
 "Carlito's Way (1993)",
 'Catwoman (2004)',
 "Charlie's Angels (2000)",
 'City Slickers (1991)',
 "City Slickers II: The Legend of Curly's Gold (1994)",
 'Clear and Present Danger (1994)',
 'Clerks (1994)',
 'Clockwork Orange, A (1971)',
 'Con Air (1997)',
 'Contact (1997)',
 'Copycat (1995)',
 'Crocodile Dundee (1986)',
 'Cruel Intentions (1999)',
 'Crying Game, The (1992)',
 'D

In [19]:
movie.columns[movie.notna().any()].to_list()

['movieId', 'title', 'genres']

#### Görev 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek

**Adım 1:** Seçilen kullanıcının izlediği fimlere ait sutunları user_movie_df'ten seçiniz ve movies_watched_df adında yeni bir dataframe oluşturunuz.

In [20]:
movies_watched_df = user_movie_df[movies_watched]

movies_watched_df.head()

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),Arachnophobia (1990),Army of Darkness (1993),Bad Santa (2003),Basic Instinct (1992),Batman Returns (1992),"Big Lebowski, The (1998)","Blair Witch Project, The (1999)","Blues Brothers, The (1980)","Boondock Saints, The (2000)","Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Carlito's Way (1993),Catwoman (2004),Charlie's Angels (2000),City Slickers (1991),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)",Con Air (1997),Contact (1997),Copycat (1995),Crocodile Dundee (1986),Cruel Intentions (1999),"Crying Game, The (1992)","Dark Crystal, The (1982)",Dawn of the Dead (2004),Dead Poets Society (1989),Deliverance (1972),Demolition Man (1993),Desperado (1995),"Devil's Advocate, The (1997)",Dirty Rotten Scoundrels (1988),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Easy Rider (1969),Edward Scissorhands (1990),Eternal Sunshine of the Spotless Mind (2004),Evil Dead II (Dead by Dawn) (1987),Ferris Bueller's Day Off (1986),Fight Club (1999),Finding Nemo (2003),Finding Neverland (2004),First Knight (1995),"Fisher King, The (1991)",Flight of the Navigator (1986),Freddy Got Fingered (2001),"Frighteners, The (1996)",From Dusk Till Dawn (1996),Gangs of New York (2002),Gattaca (1997),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),Goldfinger (1964),Goodfellas (1990),"Grand Day Out with Wallace and Gromit, A (1989)",Grease (1978),Grease 2 (1982),"Green Mile, The (1999)",Grosse Pointe Blank (1997),Harry Potter and the Prisoner of Azkaban (2004),Hero (Ying xiong) (2002),Highlander (1986),"Honey, I Shrunk the Kids (1989)","Hunt for Red October, The (1990)",Indiana Jones and the Last Crusade (1989),"Italian Job, The (2003)",Kill Bill: Vol. 1 (2003),Kill Bill: Vol. 2 (2004),Kindergarten Cop (1990),Kingpin (1996),"Knight's Tale, A (2001)",Labyrinth (1986),Last Action Hero (1993),"Last Samurai, The (2003)",Leaving Las Vegas (1995),Lemony Snicket's A Series of Unfortunate Events (2004),Liar Liar (1997),Life Is Beautiful (La Vita è bella) (1997),"Little Mermaid, The (1989)",Live and Let Die (1973),"Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Master and Commander: The Far Side of the World (2003),"Matrix, The (1999)",Meet the Parents (2000),Memento (2000),Mickey Blue Eyes (1999),Minority Report (2002),Misery (1990),Mission: Impossible (1996),Mission: Impossible II (2000),"Monsters, Inc. (2001)",Monty Python Live at the Hollywood Bowl (1982),Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mulholland Drive (2001),Muriel's Wedding (1994),My Cousin Vinny (1992),"Naked Gun: From the Files of Police Squad!, The (1988)",Nine Months (1995),Ocean's Eleven (2001),"Others, The (2001)",Pirates of the Caribbean: The Curse of the Black Pearl (2003),Police Academy 5: Assignment: Miami Beach (1988),Police Academy 6: City Under Siege (1989),"Princess Bride, The (1987)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),Ringu (Ring) (1998),Rob Roy (1995),Robin Hood: Men in Tights (1993),Romancing the Stone (1984),Rumble in the Bronx (Hont faan kui) (1995),Saving Private Ryan (1998),Scarface (1983),Schindler's List (1993),"Secret of NIMH, The (1982)",Seven (a.k.a. Se7en) (1995),Shaun of the Dead (2004),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),Shrek 2 (2004),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Snatch (2000),Sneakers (1992),So I Married an Axe Mur

In [21]:
movies_watched_df.shape

(138493, 186)

In [22]:
user_movie_df.shape

(138493, 3159)

**Adım 2:** Herbir kullancının seçili user'in izlediği filmlerin kaçını izlediği bilgisini taşıyan user_movie_count adında yeni bir dataframe oluşturunuz.

In [23]:
user_movie_count = movies_watched_df.T.notnull().sum()

user_movie_count = user_movie_count.reset_index()

user_movie_count.columns = ["userId", "movie_count"]

user_movie_count.head(5)


,userId,movie_count
0,1.0,53
1,2.0,11
2,3.0,47
3,4.0,5
4,5.0,16


**Adım 3:** Seçilen kullanıcının oy verdiği filmlerin yüzde 60 ve üstünü izleyenleri benzer kullanıcılar olarak görüyoruz. Bu kullanıcıların id’lerinden users_same_movies adında bir liste oluşturunuz.

In [24]:
perc = len(movies_watched) * 60 / 100

perc

111.6

In [25]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]

len(users_same_movies)

2326

#### Görev 4: Öneri Yapılacak Kullanıcı ile En Benzer Kullanıcıların Belirlenmesi

**Adım 1:** user_same_movies listesi içerisindeki seçili user ile benzerlik gösteren kullanıcıların id’lerinin bulunacağı şekilde movies_watched_df dataframe’ini filtreleyiniz.

In [26]:
final_df = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]

final_df.head()

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),Arachnophobia (1990),Army of Darkness (1993),Bad Santa (2003),Basic Instinct (1992),Batman Returns (1992),"Big Lebowski, The (1998)","Blair Witch Project, The (1999)","Blues Brothers, The (1980)","Boondock Saints, The (2000)","Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Carlito's Way (1993),Catwoman (2004),Charlie's Angels (2000),City Slickers (1991),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)",Con Air (1997),Contact (1997),Copycat (1995),Crocodile Dundee (1986),Cruel Intentions (1999),"Crying Game, The (1992)","Dark Crystal, The (1982)",Dawn of the Dead (2004),Dead Poets Society (1989),Deliverance (1972),Demolition Man (1993),Desperado (1995),"Devil's Advocate, The (1997)",Dirty Rotten Scoundrels (1988),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Easy Rider (1969),Edward Scissorhands (1990),Eternal Sunshine of the Spotless Mind (2004),Evil Dead II (Dead by Dawn) (1987),Ferris Bueller's Day Off (1986),Fight Club (1999),Finding Nemo (2003),Finding Neverland (2004),First Knight (1995),"Fisher King, The (1991)",Flight of the Navigator (1986),Freddy Got Fingered (2001),"Frighteners, The (1996)",From Dusk Till Dawn (1996),Gangs of New York (2002),Gattaca (1997),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),Goldfinger (1964),Goodfellas (1990),"Grand Day Out with Wallace and Gromit, A (1989)",Grease (1978),Grease 2 (1982),"Green Mile, The (1999)",Grosse Pointe Blank (1997),Harry Potter and the Prisoner of Azkaban (2004),Hero (Ying xiong) (2002),Highlander (1986),"Honey, I Shrunk the Kids (1989)","Hunt for Red October, The (1990)",Indiana Jones and the Last Crusade (1989),"Italian Job, The (2003)",Kill Bill: Vol. 1 (2003),Kill Bill: Vol. 2 (2004),Kindergarten Cop (1990),Kingpin (1996),"Knight's Tale, A (2001)",Labyrinth (1986),Last Action Hero (1993),"Last Samurai, The (2003)",Leaving Las Vegas (1995),Lemony Snicket's A Series of Unfortunate Events (2004),Liar Liar (1997),Life Is Beautiful (La Vita è bella) (1997),"Little Mermaid, The (1989)",Live and Let Die (1973),"Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Master and Commander: The Far Side of the World (2003),"Matrix, The (1999)",Meet the Parents (2000),Memento (2000),Mickey Blue Eyes (1999),Minority Report (2002),Misery (1990),Mission: Impossible (1996),Mission: Impossible II (2000),"Monsters, Inc. (2001)",Monty Python Live at the Hollywood Bowl (1982),Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mulholland Drive (2001),Muriel's Wedding (1994),My Cousin Vinny (1992),"Naked Gun: From the Files of Police Squad!, The (1988)",Nine Months (1995),Ocean's Eleven (2001),"Others, The (2001)",Pirates of the Caribbean: The Curse of the Black Pearl (2003),Police Academy 5: Assignment: Miami Beach (1988),Police Academy 6: City Under Siege (1989),"Princess Bride, The (1987)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),Ringu (Ring) (1998),Rob Roy (1995),Robin Hood: Men in Tights (1993),Romancing the Stone (1984),Rumble in the Bronx (Hont faan kui) (1995),Saving Private Ryan (1998),Scarface (1983),Schindler's List (1993),"Secret of NIMH, The (1982)",Seven (a.k.a. Se7en) (1995),Shaun of the Dead (2004),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),Shrek 2 (2004),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Snatch (2000),Sneakers (1992),So I Married an Axe Mur

In [27]:
final_df.shape

(2326, 186)

**Adım 2:** Kullanıcıların birbirleri ile olan korelasyonlarının bulunacağı yeni bir corr_df dataframe’i oluşturunuz.

In [28]:
corr_df = final_df.T.corr().unstack().sort_values()

corr_df = pd.DataFrame(corr_df, columns = ["corr"])

corr_df.index.names = ['user_id_1', 'user_id_2']

corr_df = corr_df.reset_index()

corr_df

,user_id_1,user_id_2,corr
0,110130.0,72838.0,-0.581570
1,72838.0,110130.0,-0.581570
2,33581.0,100618.0,-0.481252
3,100618.0,33581.0,-0.481252
4,126121.0,89242.0,-0.476220
...,...,...,...
5410271,91893.0,91893.0,1.000000
5410272,91867.0,91867.0,1.000000
5410273,91793.0,91793.0,1.000000
5410274,90381.0,90381.0,1.000000


In [29]:
#corr_df[corr_df["user_id_1"] == random_user]

**Adım 3:** Seçili kullanıcı ile yüksek korelasyona sahip (0.65’in üzerinde olan) kullanıcıları filtreleyerek top_users adında yeni bir dataframe oluşturunuz.

In [30]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]].reset_index(drop = True)

top_users = top_users.sort_values(by = 'corr', ascending = False)

top_users.rename(columns = {"user_id_2": "userId"}, inplace = True)

top_users.head()

,userId,corr
6,108170.0,1.000000
5,5155.0,0.716406
4,11517.0,0.707652
3,119285.0,0.700081
2,121747.0,0.673295


In [31]:
top_users.shape

(7, 2)

**Adım 4:**  top_users dataframe’ine rating veri seti ile merge ediniz.

In [32]:
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how = 'inner')

top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]

top_users_ratings["userId"].unique()

array([  5155.,  11517., 119285., 121747.,  12710.,  82860.])

In [33]:
top_users_ratings.head()

,userId,corr,movieId,rating
194,5155.0,0.716406,1,3.5
195,5155.0,0.716406,2,3.0
196,5155.0,0.716406,5,3.0
197,5155.0,0.716406,9,3.0
198,5155.0,0.716406,10,4.0


#### Görev 5: Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması

**Adım 1:** Her bir kullanıcının corr ve rating değerlerinin çarpımından oluşan weighted_rating adında yeni bir değişken oluşturunuz.

In [34]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

top_users_ratings.head()

,userId,corr,movieId,rating,weighted_rating
194,5155.0,0.716406,1,3.5,2.507423
195,5155.0,0.716406,2,3.0,2.149219
196,5155.0,0.716406,5,3.0,2.149219
197,5155.0,0.716406,9,3.0,2.149219
198,5155.0,0.716406,10,4.0,2.865626


**Adım 2:** Film id’sive her bir filme ait tüm kullanıcıların weighted rating’lerinin ortalama değerini içeren recommendation_df adında yeni bir dataframe oluşturunuz.

In [35]:
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

recommendation_df = recommendation_df.reset_index()

recommendation_df.head()

,movieId,weighted_rating
0,1,2.124196
1,2,1.319527
2,3,0.986312
3,5,1.484503
4,6,2.526375


**Adım3:** recommendation_df içerisinde weighted rating'i 3.5'ten büyük olan filmleri seçiniz ve weighted rating’e göre sıralayınız.

İlk 5 gözlemi movies_to_be_recommend olarak kaydediniz.

In [36]:
recommendation_df[recommendation_df["weighted_rating"] > 3.5]

movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending = False)

movies_to_be_recommend

,movieId,weighted_rating
229,671,3.582032
924,3098,3.582032
1349,5110,3.582032
856,2804,3.582032
1785,26840,3.538261
2183,93838,3.538261
2026,60684,3.538261
1971,53000,3.538261
1961,51559,3.538261
1838,32196,3.538261


**Adım 4:**  Tavsiye edilen 5 filmin isimlerini getiriniz.

In [37]:
movies_to_be_recommend.merge(movie[["movieId", "title"]])["title"][:5]

0    Mystery Science Theater 3000: The Movie (1996)
1                               Natural, The (1984)
2                             Super Troopers (2001)
3                         Christmas Story, A (1983)
4                       Sonatine (Sonachine) (1993)
Name: title, dtype: object

#### Görev 6: Item-Based Recommendation

Kullanıcının en son izlediği ve en yüksek puan verdiği filmin adına göre item-based öneri yapınız.

In [38]:
user = 108170

**Adım 1:** movie,rating veri setlerini okutunuz.

In [39]:
movie = pd.read_csv('datasets/movie_lens_dataset/movie.csv')
rating = pd.read_csv('datasets/movie_lens_dataset/rating.csv')

**Adım 2:** Öneri yapılacak kullanıcının 5 puan verdiği filmlerden puanı en güncel olan filmin id'sinin alınız.

In [40]:
movie_id = rating[(rating["userId"] == user) & (rating["rating"] == 5.0)].sort_values(by = "timestamp", ascending = False)["movieId"][0:1].values[0]

movie_id

7044

**Adım 3:** User based recommendation bölümünde oluşturulan user_movie_df dataframe’ini seçilen film id’sine göre filtreleyiniz.

In [41]:
movie[movie["movieId"] == movie_id]["title"].values[0]

movie_df = user_movie_df[movie[movie["movieId"] == movie_id]["title"].values[0]]

movie_df

userId
1.0        NaN
2.0        NaN
3.0        NaN
4.0        NaN
5.0        NaN
            ..
138489.0   NaN
138490.0   NaN
138491.0   NaN
138492.0   NaN
138493.0   NaN
Name: Wild at Heart (1990), Length: 138493, dtype: float64

**Adım 4:** Filtrelenen dataframe’i kullanarak seçili filmle diğer filmlerin korelasyonunu bulunuz ve sıralayınız.

In [42]:
user_movie_df.corrwith(movie_df).sort_values(ascending = False).head(10)

title
Wild at Heart (1990)                     1.000000
My Science Project (1985)                0.570187
Mediterraneo (1991)                      0.538868
Old Man and the Sea, The (1958)          0.536192
National Lampoon's Senior Trip (1995)    0.533029
Clockwatchers (1997)                     0.483337
Repo Man (1984)                          0.478674
Lost Highway (1997)                      0.476251
Blue Velvet (1986)                       0.471225
Jeffrey (1995)                           0.457849
dtype: float64

In [43]:
# Son iki adımı uygulayan fonksiyon
def item_based_recommender(movie_name, user_movie_df):
    movie = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie).sort_values(ascending = False).head(10)

**Adım 5:** Seçili film’in kendisi haricinde ilk 5 film’i öneri olarak veriniz.

In [44]:
movies_from_item_based = item_based_recommender(movie[movie["movieId"] == movie_id]["title"].values[0], user_movie_df)

# 1'den 6'ya kadar. 0'da filmin kendisi var. Onu dışarda bıraktık.

movies_from_item_based[1:6].index

Index(['My Science Project (1985)', 'Mediterraneo (1991)',
       'Old Man and the Sea, The (1958)',
       'National Lampoon's Senior Trip (1995)', 'Clockwatchers (1997)'],
      dtype='object', name='title')

In [45]:
# ['My Science Project (1985)',
# 'Mediterraneo (1991)',
# 'Old Man and the Sea,
# The (1958)',
# 'National Lampoon's Senior Trip (1995)',
# 'Clockwatchers (1997)']